In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_parquet("new.parquet", engine="fastparquet")

In [4]:
df.columns

Index(['game_id', 'turn_id', 'p1_rating', 'p2_rating', 'p1_pokemon',
       'p2_pokemon', 'p1_move', 'p2_move', 'p1_damage_taken', 'p1_status',
       'p2_damage_taken', 'p2_status', 'p1_revealed_pokemon',
       'p2_revealed_pokemon'],
      dtype='object')

In [5]:
df.dtypes

game_id                 object
turn_id                  int64
p1_rating                int64
p2_rating                int64
p1_pokemon              object
p2_pokemon              object
p1_move                 object
p2_move                 object
p1_damage_taken        float64
p1_status               object
p2_damage_taken        float64
p2_status               object
p1_revealed_pokemon     object
p2_revealed_pokemon     object
dtype: object

In [6]:
df.head()

,game_id,turn_id,p1_rating,p2_rating,p1_pokemon,p2_pokemon,p1_move,p2_move,p1_damage_taken,p1_status,p2_damage_taken,p2_status,p1_revealed_pokemon,p2_revealed_pokemon
0,2269094040-MasterRDF-TestingUnoDueTre,0,1290,1262,Breloom,Salamence,None,None,NaN,None,NaN,None,Swampert,Heracross
1,2269094040-MasterRDF-TestingUnoDueTre,1,1290,1262,Swampert,Salamence,None,Hidden Power,0.59,None,NaN,None,Swampert,Heracross
2,2269097878-KelDawntreader-loleinsmaragd,0,1362,1456,Tyranitar,Zapdos,None,None,NaN,None,NaN,None,Swampert,Flygon
3,2269097878-KelDawntreader-loleinsmaragd,1,1362,1456,Tyranitar,Flygon,Rock Slide,None,NaN,None,0.86,None,Swampert,Flygon
4,2269097878-KelDawntreader-loleinsmaragd,2,1362,1456,Swampert,Flygon,None,Toxic,0.94,tox,NaN,None,Swampert,Flygon


# Baseline Error

In [13]:
df['p2_revealed_pokemon'].value_counts().max() / len(df['p2_revealed_pokemon'])

0.07174011129493157

# General Analysis

In [15]:
print("Unique Pokemon:",len(df['p2_revealed_pokemon'].value_counts()))

Unique Pokemon: 110


In [17]:
X = df.drop('p2_revealed_pokemon', axis=1)
y = df['p2_revealed_pokemon']

In [19]:
df_number = df.select_dtypes(include='number')
df_categorical = df.select_dtypes(exclude='number')
df_categorical = pd.get_dummies(df_categorical, dtype='int')


In [20]:
df_number.corr()

,turn_id,p1_rating,p2_rating,p1_damage_taken,p2_damage_taken
turn_id,1.000000,-0.113798,-0.154085,-0.019467,-0.028947
p1_rating,-0.113798,1.000000,0.854867,0.049776,-0.017745
p2_rating,-0.154085,0.854867,1.000000,0.059692,-0.001572
p1_damage_taken,-0.019467,0.049776,0.059692,1.000000,0.210773
p2_damage_taken,-0.028947,-0.017745,-0.001572,0.210773,1.000000


It's interesting the the correlation between `p1_rating` and `p1_damage_taken` is very different from the correlation between `p2_rating` and `p2_damage_taken`.

## Looking at a subset of the data

In [30]:
subset_columns = ['p1_rating', 'p2_rating', 'p1_pokemon', 'p2_pokemon', 'p1_revealed_pokemon', 'p2_revealed_pokemon']
df_sub = df[subset_columns]

In [31]:
X_sub = df_sub.drop('p2_revealed_pokemon', axis = 1)
y_sub = df_sub['p1_revealed_pokemon']

In [32]:
X_sub_number = X_sub.select_dtypes(include='number')
X_sub_categorical = X_sub.select_dtypes(exclude='number')

In [33]:
X_sub_categorical = pd.get_dummies(X_sub_categorical, dtype='int')

In [37]:
X_sub_number.corr()

,p1_rating,p2_rating
p1_rating,1.000000,0.854867
p2_rating,0.854867,1.000000


This shows how players with similar ratings are paired up against each other